Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Onnx Pipeline

This repository shows how to deploy and use Onnx pipeline with dockers including convert model, generate input and performance test.

# Prerequisites

Pull dockers from Azure. It should take several minutes.

In [1]:
!sh build.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Error response from daemon: Get https://ziylregistry.azurecr.io/v2/: unauthorized: authentication required
Using default tag: latest
latest: Pulling from onnx-converter
Digest: sha256:59366705ce552225e0e18e17dbc353d81414809c4a1d00c9ad5fc04674a49537
Status: Image is up to date for ziylregistry.azurecr.io/onnx-converter:latest
Using default tag: latest
latest: Pulling from perf-test
Digest: sha256:983d7af9df5fba9db94cfb6cc3a04427d46387548c5c3f85b17250927b10589b
Status: Image is up to date for ziylregistry.azurecr.io/perf-test:latest


Install the onnxpipeline SDK

In [2]:
import onnxpipeline

pipeline = onnxpipeline.Pipeline()

# onnx
#pipeline = onnxpipeline.Pipeline('onnx')

# tensorflow
#pipeline = onnxpipeline.Pipeline('mnist/model')

# pytorch
#pipeline = onnxpipeline.Pipeline('pytorch')

# cntk 
#pipeline = onnxpipeline.Pipeline('cntk')

# keras
#pipeline = onnxpipeline.Pipeline('KerasToONNX')

# sklearn
#pipeline = onnxpipeline.Pipeline('sklearn')

# caffe
#pipeline = onnxpipeline.Pipeline('caffe')

# current directory
#pipeline = onnxpipeline.Pipeline()

# test mxnet fail
#pipeline = onnxpipeline.Pipeline('mxnet')


## Run parameters

(1) local_directory: string

    Required. The path of local directory where would be mounted to the docker. All operations will be executed from this path.

(2) mount_path: string

    Optional. The path where the local_directory will be mounted in the docker. Default is "/mnt/model".

(3) print_logs: boolean

    Optional. Whether print the logs from the docker. Default is True.
    
(4) convert_directory: string

    Optional. The directory path for converting model. Default is test/.    

(5) convert_name: string

    Optional. The model name for converting model. Default is model.onnx.
    


# Config information for ONNX pipeline

In [3]:
pipeline.config()

-----------config----------------
           Container information: <docker.client.DockerClient object at 0x7ff72c76fbd0>
 Local directory path for volume: /home/chuche/notebook
Volume directory path in dockers: /mnt/model
                     Result path: result
        Converted directory path: test
        Converted model filename: model.onnx
            Converted model path: test/model.onnx
        Print logs in the docker: True


# Convert model to ONNX

This image is used to convert model from major model frameworks to onnx. Supported frameworks are - caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.


You can run the docker image with customized parameters.

In [4]:

model = pipeline.convert_model(model='onnx/perf-test/ImageQnA.onnx', model_type='onnx')

# test tensorflow
#model = pipeline.convert_model(model_type='tensorflow')

# test pytorch
#model = pipeline.convert_model(model_type='pytorch', model='saved_model.pb', model_input_shapes='(1,3,224,224)')

# test cntk
#model = pipeline.convert_model(model_type='cntk', model='ResNet50_ImageNet_Caffe.model')

# test keras
#model = pipeline.convert_model(model_type='keras', model='keras_Average_ImageNet.keras')

# test sklearn
#model = pipeline.convert_model(model_type='scikit-learn', model='sklearn_svc.joblib', initial_types=("float_input", "FloatTensorType([1,4])"))

# test caffe
#model = pipeline.convert_model(model_type='caffe', model='bvlc_alexnet.caffemodel')

# test mxnet
#model = pipeline.convert_model(model_type='mxnet', model='resnet.json', model_params='resnet.params', model_input_shapes='(1,3,224,224)')



-------------

Model Conversion



Input model is already ONNX model. Skipping conversion.



-------------

MODEL INPUT GENERATION(if needed)



Input.pb already exists. Skipping dummy input generation. 



-------------

MODEL CORRECTNESS VERIFICATION





Check the ONNX model for validity 

The ONNX model is valid.



Check ONNX model for correctness. 

The original model is already onnx. Skipping correctness test. 



-------------

MODEL CONVERSION SUMMARY (.json file generated at /mnt/model/test/output.json )



{'conversion_status': 'SUCCESS',

 'correctness_verified': 'SKIPPED',

 'error_message': '',

 'input_folder': '/mnt/model/test/test_data_set_0',

 'output_onnx_path': '/mnt/model/test/model.onnx'}



## Run parameters

(1) model: string

    Required. The path of the model that needs to be converted.
    
    **IMPORTANT Only support the model path which is under the mounting directory (while initialization by Pipeline()).

(2) output_onnx_path: string

    Required. The path to store the converted onnx model. Should end with ".onnx". e.g. output.onnx

(3) model_type: string

    Required. The name of original model framework. Available types are caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.

(4) model_inputs_names: string

    (tensorflow) Optional. The model's input names. Required for tensorflow frozen models and checkpoints.

(5) model_outputs_names: string

    (tensorflow) Optional. The model's output names. Required for tensorflow frozen models checkpoints.

(6) model_params: string 

    (mxnet) Optional. The params of the model if needed.

(7) model_input_shapes: list of tuple 

    (pytorch, mxnet) Optional. List of tuples. The input shape(s) of the model. Each dimension separated by ','.

(8) target_opset: String

    Optional. Specifies the opset for ONNX, for example, 7 for ONNX 1.2, and 8 for ONNX 1.3. Defaults to '7'.
    
(9) caffe_model_prototxt: string

    (caffe) Optional. The filename of deploy prototxt for the caffe madel. 

(10) initial_types: tuple (string, string)

    (scikit-learn) Optional. A tuple consist two strings. The first is data type and the second is the size of tensor type e.g., ('float_input', 'FloatTensorType([1,4])')

(11) input_json: string

    Optional. Use JSON file as input parameters.
    
    **IMPORTANT Only support the path which is under the mounting directory (while initialization by Pipeline()).
    
(12) model_inputs_names: string

    (tensorflow) Optional.
    
(13) model_outputs_names: string

    (tensorflow) Optional.    


# Performance test tool

You can run perf_test using command python perf_test.py [Your model path] [Output path on the docker]. You can use the same arguments as for onnxruntime_pert_test tool, e.g. -m for mode, -e to specify execution provider etc. By default it will try all providers available.

In [5]:
result = pipeline.perf_test(model=model)
#result = pipeline.perf_test()   # is ok, too

Setting thread pool size to 1

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-ten

Total time cost:0.0200326

Total iterations:20

Average time cost:1.00163 ms

Setting thread pool size to 4

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_util

Total time cost:0.50956

Total iterations:20

Average time cost:25.478 ms

Setting thread pool size to 2

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.h

Total time cost:0.469658

Total iterations:20

Average time cost:23.4829 ms

Setting thread pool size to 1

Total time cost:1.27837

Total iterations:20

Average time cost:63.9187 ms

Setting thread pool size to 4

Total time cost:1.30063

Total iterations:20

Average time cost:65.0315 ms

Setting thread pool size to 5

Total time cost:1.47124

Total iterations:20

Average time cost:73.5621 ms

Setting thread pool size to 6

Total time cost:0.54477

Total iterations:20

Average time cost:27.2385 ms

2019-07-09 18:47:55.409884146 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape__72

2019-07-09 18:47:55.410065503 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spa

Setting thread pool size to 1

Total time cost:0.423756

Total iterations:20

Average time cost:21.1878 ms

2019-07-09 18:47:59.217477099 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape__72

2019-07-09 18:47:59.217542055 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape_1__64

2019-07-09 18:47:59.217568253 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/WordCo

Setting thread pool size to 4

Total time cost:0.391039

Total iterations:20

Average time cost:19.552 ms

2019-07-09 18:48:02.441177921 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape__72

2019-07-09 18:48:02.441226650 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape_1__64

2019-07-09 18:48:02.441254335 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/WordCon

Setting thread pool size to 5

Total time cost:0.421863

Total iterations:20

Average time cost:21.0931 ms

Setting thread pool size to 0

Total time cost:0.0202769

Total iterations:20

Average time cost:1.01384 ms

Setting thread pool size to 0

Total time cost:0.0320109

Total iterations:20

Average time cost:1.60054 ms

Setting thread pool size to 0

Total time cost:0.0366491

Total iterations:20

Average time cost:1.83246 ms

Setting thread pool size to 0

Total time cost:0.0254549

Total iterations:20

Average time cost:1.27275 ms

Setting thread pool size to 0

Total time cost:0.0306197

Total iterations:20

Average time cost:1.53098 ms

Setting thread pool size to 0

Total time cost:0.0318539

Total iterations:20

Average time cost:1.59269 ms

Setting thread pool size to 0

Total time cost:0.0209091

Total iterations:20

Average time cost:1.04545 ms

Setting thread pool size to 0

Total time cost:0.0237088

Total iterations:20

Average time cost:1.18544 ms

Setting thread pool 

/perf_test/bin/RelWithDebInfo/cpu/onnxruntime_perf_test -x 1 -m times -r 20 /mnt/model/test/model.onnx /mnt/model/result/48e434c7-d869-44ca-8395-c9735f498ad1Setting thread pool size to 4

Total time cost:7.51595

Total iterations:20

Average time cost:375.797 ms

Setting thread pool size to 2

Total time cost:1.1945

Total iterations:20

Average time cost:59.725 ms

Setting thread pool size to 3

Total time cost:0.0186974

Total iterations:20

Average time cost:0.934872 ms

Setting thread pool size to 1

Total time cost:11.5703

Total iterations:20

Average time cost:578.514 ms

Setting thread pool size to 4

Total time cost:11.1356

Total iterations:20

Average time cost:556.78 ms

Setting thread pool size to 2

Total time cost:11.2746

Total iterations:20

Average time cost:563.729 ms

Setting thread pool size to 0

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_reposi

Total time cost:0.495737

Total iterations:20

Average time cost:24.7868 ms

Setting thread pool size to 0

Total time cost:0.142478

Total iterations:20

Average time cost:7.12388 ms

2019-07-09 18:50:26.384164975 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape__72

2019-07-09 18:50:26.384217955 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape_1__64

2019-07-09 18:50:26.384243519 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_l

Setting thread pool size to 0

Total time cost:0.0199557

Total iterations:20

Average time cost:0.997783 ms

Setting thread pool size to 0

Total time cost:0.0211771

Total iterations:20

Average time cost:1.05886 ms

Setting thread pool size to 0

Total time cost:0.0546229

Total iterations:20

Average time cost:2.73115 ms

Setting thread pool size to 0

Total time cost:0.0227626

Total iterations:20

Average time cost:1.13813 ms

Setting thread pool size to 0

Total time cost:18.8279

Total iterations:20

Average time cost:941.396 ms

Setting thread pool size to 0

Total time cost:0.0741294

Total iterations:20

Average time cost:3.70647 ms

Setting thread pool size to 0

Total time cost:7.56272

Total iterations:20

Average time cost:378.136 ms

Setting thread pool size to 0

Total time cost:0.0217053

Total iterations:20

Average time cost:1.08526 ms

Setting thread pool size to 0

Total time cost:0.0189598

Total iterations:20

Average time cost:0.947992 ms

Setting thread pool s

# Run parameters

(1) model: string

    Required. The path of the model that wants to be performed.
    
(2) result: string

    Optional. The path of the result.
    
(3) config: string (choices=["Debug", "MinSizeRel", "Release", "RelWithDebInfo"])

    Optional. Configuration to run. Default is "RelWithDebInfo".
    
(4) mode: string (choices=["duration", "times"])

    Optional. Specifies the test mode. Value could be 'duration' or 'times'. Default is "times".

(5) execution_provider: string (choices=["cpu", "cuda", "mkldnn"])

    Optional. help="Specifies the provider 'cpu','cuda','mkldnn'. Default is ''.
    
(6) repeated_times: integer

    Optional. Specifies the repeated times if running in 'times' test mode. Default:20.
    
(7) duration_times: integer

    Optional. Specifies the seconds to run for 'duration' mode. Default:10.
    
(8) parallel: boolean

    Optional. Use parallel executor, default (without -x): sequential executor.
    
(9) threadpool_size: integer

    Optional. Threadpool size if parallel executor (--parallel) is enabled. Default is the number of cores.
    
(10) num_threads: integer

    Optional. OMP_NUM_THREADS value.
    
(11) top_n: integer

    Optional. Show percentiles for top n runs. Default:5.
    
(12) runtime: boolean

    Optional. Use this boolean flag to enable GPU if you have one.
    
(13) input_json: string

    Optional. Use JSON file as input parameters.
   

In [6]:
pipeline.print_performance(result)
#pipeline.print_result() # is ok, too

mkldnn_openmp_6_threads_OMP_WAIT_POLICY_active 0.000596571 s

mkldnn_openmp_6_threads 0.000648833 s

cpu_openmp_5_threads_OMP_WAIT_POLICY_active 0.00066767 s

mkldnn_openmp_5_threads 0.000669885 s

mkldnn_openmp_5_threads_OMP_WAIT_POLICY_active 0.000670425 s

cpu_openmp_5_threads 0.000781311 s

cpu_openmp_6_threads_OMP_WAIT_POLICY_active 0.000831343 s

cpu 0.000868031 s

cpu_openmp_6_threads 0.000917568 s

cuda_5_threads 0.000918293 s

cuda 0.000922196 s

cpu_openmp_4_threads_OMP_WAIT_POLICY_active 0.000922242 s

cuda_6_threads 0.000923248 s

tensorrt 0.000927788 s

cuda_4_threads 0.000929841 s

cpu_openmp_4_threads 0.000932158 s

cuda_1_threads 0.000934594 s

tensorrt_1_threads 0.000937554 s

mkldnn_openmp_4_threads 0.000937782 s

tensorrt_2_threads 0.000938546 s

tensorrt_4_threads 0.000940796 s

mkldnn_openmp_4_threads_OMP_WAIT_POLICY_active 0.000950036 s

cpu_openmp 0.000964055 s

ngraph 0.000990395 s

ngraph_4_threads 0.000992804 s

cpu_openmp_1_threads_OMP_WAIT_POLICY_passive 0.0

# Result Visualization

After performance test, there would be a directory for results. 

This libray use Pandas.read_json to visualize JSON file. (orient is changeable.)

"latency.json" contains the raw data of results ordered by the average time. 

Use .latency to obtain the original latency JSON; Use .profiling to obtain original top 5 profiling JSON.

In [7]:
r = pipeline.get_result(result)
#r.latency
#r.profiling

# Print latency.json

Provide parameters for top 5 performace. Use the parameter "top" to visualize more results.

In [8]:
r.prints()

,name,avg,p90,p95,cpu_usage,gpu_usage,memory_util,code_snippet
0,mkldnn_openmp_6_threads_OMP_WAIT_POLICY_active,0.000597,0.000677,0.000788,0.554250,0,0.041937,"OrderedDict([(u'execution_provider', u''), (u'..."
1,mkldnn_openmp_6_threads,0.000649,0.000761,0.000857,0.499588,0,0.041940,"OrderedDict([(u'execution_provider', u''), (u'..."
2,cpu_openmp_5_threads_OMP_WAIT_POLICY_active,0.000668,0.000742,0.000783,0.520371,0,0.041994,"OrderedDict([(u'execution_provider', u''), (u'..."
3,mkldnn_openmp_5_threads,0.000670,0.000769,0.000790,0.527400,0,0.041949,"OrderedDict([(u'execution_provider', u''), (u'..."
4,mkldnn_openmp_5_threads_OMP_WAIT_POLICY_active,0.000670,0.000734,0.000785,0.519000,0,0.041952,"OrderedDict([(u'execution_provider', u''), (u'..."


# Print profiling.json

Only provide profiling JSON for top 5 performace by giving certain index of the result. The file name is profile_[name].json
(1) index: integer
    Required. The index for top 5 profiling files.
(2) top: integer
    The number for top Ops.


In [9]:
r.print_profiling(index=4, top=5)

,cat,dur,name,op_name,ph,pid,provider,tid,ts
0,Session,50687,model_loading_uri,NaN,X,1712,NaN,1712,1
1,Session,51895,session_initialization,NaN,X,1712,NaN,1712,50695
2,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Gather,X,1712,NaN,1712,102750
3,Node,531,agi.lib.tf.model_repository.spacev_co_train_no...,Gather,X,1712,CPUExecutionProvider,1712,102753
4,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Gather,X,1712,NaN,1712,103287


# Get environment_variables in code_snippet

Given the index from latency.json, output the environment variables.

In [10]:
r.print_environment(3)

,0
LD_LIBRARY_PATH,/perf_test/bin/RelWithDebInfo/mkldnn_openmp


# Get codes

Given the index from latency.json, output the codes.

In [11]:
print(r.get_code(3))


import onnxruntime as ort
so = rt.SessionOptions()
so.set_graph_optimization_level(2)
so.enable_sequential_execution = True
so.session_thread_pool_size(0)
session = rt.Session("/mnt/model/test/model.onnx", so)



# netron

In [ ]:
def json_to_csv(self, json_name, erase_keys):
    with open(json_name) as json_file:  
        json_data = json.load(json_file, object_pairs_hook=OrderedDict) 

    csv_name = 'temp.csv'
    with open(csv_name, 'w') as f:
        writer = csv.writer(f)
        keys = []
        for key in json_data[0]:
            if key not in erase_keys:
                keys.append(key)
        writer.writerow(keys)
        for j in json_data:
            values = []
            for key in j:
                if key not in erase_keys:
                    values.append(j[key])
            writer.writerow(values)
        return pd.read_csv(csv_name) 